In [1]:
from flyvis.datasets.moving_bar import MovingEdge
import numpy as np
import os
from flyvis_cell_type_pert import FlyvisCellTypePert
import pandas as pd
from tqdm import tqdm
from moving_edge import MovingEdgeWrapper
from pathlib import Path
from flyvis_cell_type_pert import PerturbationType

 -> Patched datamate.directory._write_h5
Importing flyvis...


In [2]:
import datamate

data_path = Path("data/flyvis_data")
data_path.mkdir(parents=True, exist_ok=True)

env = os.environ.copy()
env["FLYVIS_ROOT_DIR"] = str(data_path)


# def fixed_write_h5(path, val):
#     """
#     A Windows-safe replacement that skips the 'read-before-write' check.
#     """
#     path = Path(path)
#     path.parent.mkdir(parents=True, exist_ok=True)

#     # Simple, safe write. No checking 'f["data"]', no 'unlink'.
#     # This completely bypasses the logic causing your KeyError.
#     with h5py.File(path, mode="w", libver="latest") as f:
#         f.create_dataset("data", data=val)
# datamate.io._write_h5 = fixed_write_h5
# if hasattr(datamate.directory, "_write_h5"):
#     datamate.directory._write_h5 = fixed_write_h5
#     print(" -> Patched datamate.directory._write_h5")
# else:
#     print(" -> Warning: Could not find _write_h5 in directory module")


In [3]:
cell_type_df = pd.read_csv(f'{data_path}/flyvis_cell_type_connectivity.csv')

dataset = MovingEdge(
            offsets=[-10, 11],  # offset of bar from center in 1 * radians(2.25) led size
            intensities=[0, 1],  # intensity of bar
            speeds=[19],  # speed of bar in 1 * radians(5.8) / s
            height=80,  # height of moving bar in 1 * radians(2.25) led size
            post_pad_mode="continue",  # for post-stimulus period, continue with the last frame of the stimulus
            t_pre=1.0,  # duration of pre-stimulus period
            t_post=1.0,  # duration of post-stimulus period
            dt=1 / 200,  # temporal resolution of rendered video
            angles=list(np.arange(0, 360, 30)),  # motion direction (orthogonal to edge)
    )

### pair - wise pert

In [4]:
split_dfs = np.array_split(cell_type_df, 3)
df_part1 = split_dfs[0]
df_part2 = split_dfs[1]
df_part3 = split_dfs[2]

c:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
# wrapper = MovingEdgeWrapper(dataset, pert=None, pert_folder_name=None,
#                                 output_file_name="data/flyvis_data/mv_edge_original_network.csv")
# wrapper.run()

In [6]:
for src, tar in tqdm(cell_type_df[['source_type', 'target_type']].values):
    result_output_path = f'data/flyvis_data/pairwise_edge_pert_results/{src}_{tar}.csv'
    if os.path.exists(result_output_path):
        print(f"Skipping existing perturbation: {src} -> {tar}")
        continue
    print(f"Running perturbation: {src} -> {tar}")

    pert = FlyvisCellTypePert()
    pert.perturb(cell_type_df, PerturbationType.PAIR_WISE, pairs=[(src, tar)])

    wrapper = MovingEdgeWrapper(dataset, pert=pert,
                                pert_folder_name=f'{src}_{tar}_perturbation',
                                output_file_name=result_output_path)
    wrapper.run()

100%|██████████| 604/604 [00:00<00:00, 29545.62it/s]

Skipping existing perturbation: Am -> C2
Skipping existing perturbation: Am -> C3
Skipping existing perturbation: Am -> L1
Skipping existing perturbation: Am -> L2
Skipping existing perturbation: Am -> L3
Skipping existing perturbation: Am -> L4
Skipping existing perturbation: Am -> R1
Skipping existing perturbation: Am -> R2
Skipping existing perturbation: Am -> R4
Skipping existing perturbation: Am -> R5
Skipping existing perturbation: Am -> T1
Skipping existing perturbation: C2 -> Am
Skipping existing perturbation: C2 -> L1
Skipping existing perturbation: C2 -> L2
Skipping existing perturbation: C2 -> L3
Skipping existing perturbation: C2 -> L4
Skipping existing perturbation: C2 -> L5
Skipping existing perturbation: C2 -> Lawf2
Skipping existing perturbation: C2 -> Mi1
Skipping existing perturbation: C2 -> T1
Skipping existing perturbation: C2 -> T2
Skipping existing perturbation: C2 -> T4a
Skipping existing perturbation: C2 -> Tm1
Skipping existing perturbation: C2 -> Tm9
Skipping 

### outgoing pert

In [7]:
import os
from tqdm import tqdm
# Assuming these are the correct imports for your framework
# from flyvis import FlyvisCellTypePert, PerturbationType, MovingEdgeWrapper

# 1. Ensure the output directory exists to avoid FileNotFoundError
output_dir = 'data/flyvis_data/outgoing_edge_pert_moving_edge_results'
os.makedirs(output_dir, exist_ok=True)

# Loop through unique source types
for src in tqdm(cell_type_df['source_type'].unique()):
    
    result_output_path = os.path.join(output_dir, f'{src}_outgoing.csv')
    
    # 2. Check if the file already exists to skip redundant work
    if os.path.exists(result_output_path):
        print(f"Skipping existing perturbation: {src} outgoing")
        continue
        
    print(f"Running perturbation: {src} outgoing")

    # 3. Initialize the perturbation object
    pert = FlyvisCellTypePert()
    
    # Ensure PerturbationType.OUTGOING is defined in your imports
    pert.perturb(
        cell_type_df, 
        PerturbationType.OUTGOING, 
        source_outgoing=src
    )

    # 4. Initialize and run the wrapper
    # Ensure 'dataset' is already defined in your environment
    wrapper = MovingEdgeWrapper(
        dataset, 
        pert=pert,
        pert_folder_name=f'{src}_outgoing_perturbation',
        output_file_name=result_output_path
    )
    
    wrapper.run()

100%|██████████| 63/63 [00:00<00:00, 15687.55it/s]

Skipping existing perturbation: Am outgoing
Skipping existing perturbation: C2 outgoing
Skipping existing perturbation: C3 outgoing
Skipping existing perturbation: CT1(Lo1) outgoing
Skipping existing perturbation: CT1(M10) outgoing
Skipping existing perturbation: L1 outgoing
Skipping existing perturbation: L2 outgoing
Skipping existing perturbation: L3 outgoing
Skipping existing perturbation: L4 outgoing
Skipping existing perturbation: L5 outgoing
Skipping existing perturbation: Lawf1 outgoing
Skipping existing perturbation: Lawf2 outgoing
Skipping existing perturbation: Mi1 outgoing
Skipping existing perturbation: Mi10 outgoing
Skipping existing perturbation: Mi12 outgoing
Skipping existing perturbation: Mi13 outgoing
Skipping existing perturbation: Mi14 outgoing
Skipping existing perturbation: Mi15 outgoing
Skipping existing perturbation: Mi2 outgoing
Skipping existing perturbation: Mi3 outgoing
Skipping existing perturbation: Mi4 outgoing
Skipping existing perturbation: Mi9 outgoing

### motif pert

In [ ]:
output_dir = 'data/flyvis_data/motifs_pert_moving_edge_results'
os.makedirs(output_dir, exist_ok=True)

pert = FlyvisCellTypePert()
motif_id = "78 ['+', '+', '-', '-']"
pert.perturb(cell_type_df, PerturbationType.MOTIF, motif_id=str(motif_id))

wrapper = MovingEdgeWrapper(
    dataset,
    pert=pert,
    pert_folder_name=f'{motif_id}_motif_perturbation',
    output_file_name=os.path.join(output_dir, f'{motif_id}_motif.csv')
)

wrapper.run()

[2026-01-07 19:11:10] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation
[2026-01-07 19:11:10] logging_utils:23 epe not in C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation\validation, but 'loss' is. Falling back to 'loss'. You can rerun the ensemble validation to make appropriate recordings of the losses.
[2026-01-07 19:11:17] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 19:11:17] chkpt_utils:36 Recovered network state.


Running moving edge simulation...
Applying perturbation to network in memory...


[2026-01-07 19:11:18] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation


Overwriting disk checkpoints with perturbed weights...
 -> Updated: data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation\best_chkpt
 -> Updated: data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation\chkpts\chkpt_00000
Clearing caches...
 -> Removed __cache__
Generating moving edge responses...


[2026-01-07 19:11:25] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 19:11:25] chkpt_utils:36 Recovered network state.
[2026-01-07 19:11:27] network:757 Computing 24 stimulus responses.


Batch:   0%|          | 0/6 [00:00<?, ?it/s]

[2026-01-07 19:12:43] xarray_joblib_backend:56 Store item C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_motif_perturbation\__cache__\flyvis\analysis\stimulus_responses\compute_responses\2f0bd6f778a8fda21b02aaa7f52e3d75\output.h5


Evaluating performance...


In [ ]:
edges_pert = pert.pert_conn[pert.pert_conn.pert_weight == 0]
number_of_edges_pert = len(edges_pert)
print(f"Number of edges perturbed in motif {motif_id}: {number_of_edges_pert}")
# edges_pert.sign.value_counts()

random_edges = cell_type_df.sample(n=number_of_edges_pert, random_state=42)
randon_pairs = list(zip(random_edges.source_type, random_edges.target_type))

Number of edges perturbed in motif 78 ['+', '+', '-', '-']: 112


sign
-1.0    56
 1.0    56
Name: count, dtype: int64

In [20]:
pert = FlyvisCellTypePert()
pert.perturb(cell_type_df, PerturbationType.PAIR_WISE, pairs=randon_pairs)

wrapper = MovingEdgeWrapper(
    dataset,
    pert=pert,
    pert_folder_name=f'{motif_id}_null_perturbation',
    output_file_name=os.path.join(output_dir, f'{motif_id}_null.csv')
)

wrapper.run()

[2026-01-07 19:21:17] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation
[2026-01-07 19:21:17] logging_utils:23 epe not in C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation\validation, but 'loss' is. Falling back to 'loss'. You can rerun the ensemble validation to make appropriate recordings of the losses.
[2026-01-07 19:21:21] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 19:21:21] chkpt_utils:36 Recovered network state.


Running moving edge simulation...
Applying perturbation to network in memory...


[2026-01-07 19:21:21] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation


Overwriting disk checkpoints with perturbed weights...
 -> Updated: data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation\best_chkpt
 -> Updated: data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation\chkpts\chkpt_00000
Clearing caches...
 -> Removed __cache__
Generating moving edge responses...


[2026-01-07 19:21:25] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-07 19:21:25] chkpt_utils:36 Recovered network state.
[2026-01-07 19:21:27] network:757 Computing 24 stimulus responses.


Batch:   0%|          | 0/6 [00:00<?, ?it/s]

[2026-01-07 19:22:33] xarray_joblib_backend:56 Store item C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_78 ['+', '+', '-', '-']_null_perturbation\__cache__\flyvis\analysis\stimulus_responses\compute_responses\32340e65d1e3b22b781586cafbc92027\output.h5


Evaluating performance...
